In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/Users/tj/dcu/Group-Project


# Data

In [3]:
from src.features.basic import features
from src.features.common import postproc_default
from src.dataset.tc4tl import make_datasets
from src.hyperopt import RFP_PARAMS

In [17]:
trainset, devset, testset = make_datasets(features, postproc_default, tunables=RFP_PARAMS, verbose=True, testing=100)

100%|██████████| 100/100 [00:00<00:00, 588.83it/s]


In [15]:
trainset

,RssiMin,RssiMax,RssiMean,CoarseGrain,fileid,Distance,DistanceFloat,RssiMinDistance,RssiMaxDistance,RssiMeanDistance
0,-96.0,-65.0,-76.373585,1,ujopbzgg_tc4tl20.csv,4.5,4.5,39.760451,1.520493,5.035454
1,-98.0,-49.0,-64.356547,0,cayohaoh_tc4tl20.csv,4.5,4.5,49.079573,0.282091,1.420897
2,-76.0,-51.0,-58.155763,1,pvnregmh_tc4tl20.csv,1.8,1.8,4.841239,0.348208,0.739662
3,-91.0,-61.0,-73.291252,0,lipprkdy_tc4tl20.csv,4.5,4.5,23.487004,0.997897,3.639987
4,-92.0,-53.0,-65.235140,0,akyvvxvo_tc4tl20.csv,4.5,4.5,26.094703,0.429821,1.558605
...,...,...,...,...,...,...,...,...,...,...
95,-91.0,-59.0,-67.427136,0,optadgkr_tc4tl20.csv,4.5,4.5,23.487004,0.808418,1.963200
96,-80.0,-56.0,-66.090598,0,rvebtbzu_tc4tl20.csv,4.5,4.5,7.376588,0.589470,1.705500
97,-91.0,-56.0,-72.453841,0,kvfqiqnz_tc4tl20.csv,1.8,1.8,23.487004,0.589470,3.332801
98,-96.0,-57.0,-66.962183,1,urbzdqte_tc4tl20.csv,4.5,4.5,39.760451,0.654917,1.869411


## Evaluate Basic Feature

In [5]:
devset.shape, testset.shape

((935, 7), (8423, 7))

In [6]:
from src.models.base import dual_evaluation
import pandas as pd

f1_report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=["RssiMin", "RssiMax", "RssiMean", "CoarseGrain"],
                    target="Distance")

In [77]:
print(f1_report['cg=0'])

              precision    recall  f1-score   support

         1.8       0.74      0.74      0.74      1423
         4.5       0.74      0.73      0.74      1401

    accuracy                           0.74      2824
   macro avg       0.74      0.74      0.74      2824
weighted avg       0.74      0.74      0.74      2824



In [78]:
print(f1_report['cg=1'])

              precision    recall  f1-score   support

         1.2       0.55      0.58      0.57      1407
         1.8       0.42      0.45      0.43      1408
         3.0       0.38      0.38      0.38      1385
         4.5       0.49      0.43      0.46      1399

    accuracy                           0.46      5599
   macro avg       0.46      0.46      0.46      5599
weighted avg       0.46      0.46      0.46      5599



In [79]:
from src.ndcf import compute_ndcf

ndcf = compute_ndcf(sysout)
ndcf

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.417910,0.160544,0.578454
1,fine_grain,1.8,0.286323,0.355963,0.642286
2,fine_grain,3.0,0.150476,0.571122,0.721598
3,coarse_grain,1.8,0.255095,0.266238,0.521333


In [80]:
ndcf["nDCF"].mean()

0.6159179861376931

## Add Linear Approximation Distance Estimate as feature

In [60]:
from src.features import radioprop
from src.featutils import compose
from src.features.common import postproc_basic

def f2(feats, tunables={}):
    for name in ["RssiMin", "RssiMax", "RssiMean"]:
        rssi = feats[name]
        distance = radioprop.linear_approximation_model(rssi, **tunables["LinearApprox"])
        feats[name + "Distance"] = distance
    return feats

trainset, devset, testset = make_datasets(compose(features, f2), postproc_basic,
                                          tunables=RFP_PARAMS, verbose=True, testing=0)

100%|██████████| 8423/8423 [00:11<00:00, 747.50it/s]


In [61]:
trainset.head()

,RssiMin,RssiMax,RssiMean,RssiMinDistance,RssiMaxDistance,RssiMeanDistance,CoarseGrain,fileid,Distance,DistanceFloat
0,0.383333,0.128713,0.343245,0.087135,0.064738,0.139819,0,aaadbuvp_tc4tl20.csv,1.8,1.8
1,0.583333,0.133663,0.470944,0.023335,0.058268,0.065995,1,aaamkcii_tc4tl20.csv,3.0,3.0
2,0.366667,0.138614,0.340356,0.097011,0.052445,0.142186,1,aabqtowt_tc4tl20.csv,1.2,1.2
3,0.300000,0.108911,0.235470,0.148762,0.098641,0.260539,1,aadkjwss_tc4tl20.csv,1.8,1.8
4,0.650000,0.148515,0.461564,0.014693,0.042487,0.069789,0,aafzrgzt_tc4tl20.csv,1.8,1.8


### Evaluate

In [72]:
from src.models.base import dual_evaluation
import pandas as pd

# predictors = [ col for col in trainset.columns if 'rssi' in col.lower() or 'coarse' in col.lower() ]
predictors = ["RssiMin", "RssiMax", "RssiMean", "RssiMeanDistance" , "RssiMinDistance", "RssiMaxDistance"]
f1_report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors, target="Distance")

In [73]:
print(f1_report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.74      0.74      0.74      1423
         4.5       0.74      0.73      0.73      1401

    accuracy                           0.74      2824
   macro avg       0.74      0.74      0.74      2824
weighted avg       0.74      0.74      0.74      2824



In [74]:
print(f1_report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.55      0.58      0.57      1407
         1.8       0.41      0.44      0.42      1408
         3.0       0.39      0.39      0.39      1385
         4.5       0.50      0.44      0.47      1399

    accuracy                           0.46      5599
   macro avg       0.46      0.46      0.46      5599
weighted avg       0.46      0.46      0.46      5599



In [75]:
from src.ndcf import compute_ndcf

ndcf = compute_ndcf(sysout)
ndcf

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.420043,0.158635,0.578678
1,fine_grain,1.8,0.289876,0.360632,0.650508
2,fine_grain,3.0,0.147619,0.561115,0.708734
3,coarse_grain,1.8,0.260014,0.266238,0.526252


In [76]:
ndcf["nDCF"].mean()

0.6160431439663822